In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
import os
import shutil
import random
import gym
import cv2
import math
from PIL import Image
import pandas as pd
from scipy.stats import norm
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import clip

In [ ]:
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
pipe.load_lora_weights("lora weights/unique_artist.safetensors", weight_name="pytorch_lora_weights.safetensors")
pipe = pipe.to("cuda")

In [ ]:
def get_prompt(random):
    prompts = [
        f"Create an image of a working on a tour plan in a",
        f"Create an image of a brainstorming new ideas in a",
        f"Create an image of a actively working on a project in a",
        f"Create an image of a reflecting on their work in a",
        f"Create an image of a collaborating with colleagues in a",
        f"Create an image of a teaching or presenting in a",
        f"Create an image of a conducting research in a",
        f"Create an image of a creating an art piece in a",
        f"Create an image of a solving a complex problem in a",
        f"Create an image of a giving a speech or a lecture in a",
        f"Create an image of a experimenting with new techniques in a"
        f"Create an image of a designing a new invention in a",
        f"Create an image of a leading a team meeting in a",
        f"Create an image of a analyzing data on a computer in a",
        f"Create an image of a writing a book in a",
        f"Create an image of a gardening in a",
        f"Create an image of a playing a musical instrument in a",
        f"Create an image of a practicing yoga in a",
        f"Create an image of a cooking in a gourmet kitchen in a",
        f"Create an image of a building a robot in a",
        f"Create an image of a exploring a historic site in a"
    ]
    return prompts[random]

descriptions = ['unique', 'distinctive', 'cool']  
professions = ['scientist', 'artist', 'professor']  
settings = ['corporate office', 'research center', 'classroom'] 

def create_prompt(a,b,c, prompt):
    sentence = prompt[:21] + descriptions[a] + " " + professions[b] + " " + prompt[21:] + ' ' +settings[c]+ '.'
    return sentence


In [ ]:
NUM_ACTION = 3


# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)

In [ ]:
class StablediffusionEnv(gym.Env):
    def __init__(self, pipe):
        super(StablediffusionEnv, self).__init__()
        self.prompt = None
        self.current_word = None
        embedding_size = 512
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(embedding_size,), dtype=np.float32)
        self.action_space = gym.spaces.Discrete((NUM_ACTION)**3)  
        self.prompt_embed = None      
        
        self.action_dict = {}
        count = 0
        for i in range(NUM_ACTION):
            for j in range(NUM_ACTION):
                for k in range(NUM_ACTION):
                    self.action_dict[count] = (i, j, k)
                    count += 1

    def reset(self):
        self.current_word = None
        self.prompt = get_prompt(random.randint(0, 19))
        prompt_clip_text = clip.tokenize([self.prompt]).to(device)  # Ensure the device is consistent
        self.prompt_embed = clip_model.encode_text(prompt_clip_text)
        return self.prompt_embed.cpu().detach().numpy()  # Convert to numpy after moving to CPU



    def step(self, action):
        action_count[action] += 1
        word1, word2, word3 = self.action_dict[action]
        self.current_word = create_prompt(word1,word2,word3,self.prompt)
        print(self.current_word)

        prediction = pipe([self.current_word]).images
        
        # plt.imshow(prediction[0])
        # plt.axis('off')  
        # plt.show()
        
        #clip embeddings
        clip_text = clip.tokenize([self.current_word]).to(device)
        
        clip_image = clip_preprocess(prediction[0]).unsqueeze(0).to(device)
        
        with torch.no_grad():
            image_features = clip_model.encode_image(clip_image)
            text_features = clip_model.encode_text(clip_text)

        
        cosine_similarity = torch.nn.functional.cosine_similarity(text_features, image_features, dim=1)
        
        reward = 100*(1-cosine_similarity.item())
        print('Reward', {reward})
        self.last_images = np.array(prediction[0])
        done = True
        return np.array(prediction[0]).mean(axis=2).mean(axis=1), reward, done, {'image' : prediction[0]}

In [ ]:
env = DummyVecEnv([lambda: StablediffusionEnv(pipe)])
dqn_model = DQN("MlpPolicy", env, buffer_size=10000, verbose=1, exploration_final_eps=0.6, exploration_initial_eps=1.0)

In [ ]:
# Training

action_count = [0] * (NUM_ACTION**3)
dqn_model.learn(3000)

# # dqn_model.save('dqn_model_lora_SDv4_3000')

In [ ]:
dqn_model = DQN.load('RL models/dqn_model_lora_SDv4_3000.zip')

In [ ]:
# Get mean probabilities

proxy_prob_clip = []

for episode in range(1000):
    done = False
    obs = env.reset()
    # get Q values
    q_values = dqn_model.policy.q_net(torch.tensor(obs, dtype=torch.float32, device = dqn_model.device))
    action_probabilities = torch.nn.functional.softmax(q_values, dim=1)
    proxy_prob_clip.append(action_probabilities)

concatenated_data = torch.cat(proxy_prob_clip, dim=0)
transposed_data = concatenated_data.t()

proxy_mean = torch.mean(transposed_data, dim=1)
proxy_mean

In [ ]:
episode_rewards = []
episode_actions = []
proxy_prob = []

store_img = [[] for _ in range(NUM_ACTION**3)]
action_count = [0] * (NUM_ACTION**3)
for episode in range(500):
    print(f"Episode: {episode}")
    done = False
    obs = env.reset()
    q_values = dqn_model.policy.q_net(torch.tensor(obs, dtype=torch.float32, device = dqn_model.device))
    action_probabilities = torch.nn.functional.softmax(q_values, dim=1)
    proxy_prob.append(action_probabilities)
    while not done:
        action, _ = dqn_model.predict(obs)
        obs, reward, done, info = env.step(action)
        store_img[action[0]].append(info[0]['image'])
        episode_rewards.append(reward)
        episode_actions.append(action)

In [ ]:
print(episode_rewards)
print(episode_actions)

In [ ]:
# Save images

list_of_lists_of_images = store_img

base_dir = 'saved_images_lora'
os.makedirs(base_dir, exist_ok=True)

with open('image_paths.txt', 'w') as file:
    for i, sublist in enumerate(list_of_lists_of_images):
        sublist_dir = os.path.join(base_dir, f'sublist_{i}')
        os.makedirs(sublist_dir, exist_ok=True)

        for j, np_image in enumerate(sublist):
            # Convert NumPy array to PIL Image
            pil_image = Image.fromarray(np.uint8(np_image))
            image_path = os.path.join(sublist_dir, f'image_{j}.png')
            pil_image.save(image_path)

            # Write the path to the file
            file.write(image_path + '\n')


In [ ]:
plt.plot(episode_rewards)
plt.xlabel('steps')
plt.ylabel('reward')


In [ ]:
unique_actions, action_counts = np.unique(np.concatenate(episode_actions), return_counts=True)
plt.bar(unique_actions, action_counts, color="green")
plt.xlabel('Action')
plt.ylabel('Frequency')
plt.title('Stable diffusion v1-4')
plt.tight_layout()
plt.show()

In [ ]:
action_dict2 = {}
count2 = 0
for i in range(NUM_ACTION):
    for j in range(NUM_ACTION):
        for k in range(NUM_ACTION):
            action_dict2[count2] = (i, j, k)
            count2 += 1
action_dict2[9]

In [ ]:
concatenated_data = torch.cat(proxy_prob, dim=0)
transposed_data = concatenated_data.t()

proxy_mean = torch.mean(transposed_data, dim=1)
proxy_std = torch.std(transposed_data, dim = 1)

proxy_std_log = torch.log(proxy_std)
proxy_std_log_np = proxy_std_log.cpu().detach().numpy()
sorted_indices = np.argsort(proxy_std_log_np)
ranks = np.empty_like(sorted_indices)
ranks[sorted_indices] = np.arange(len(proxy_std_log_np))

ranks+=1

ranks_tensor = torch.tensor(ranks)

proxy_mean = proxy_mean.reshape(3,3,3).cpu().detach().numpy()
scale = (ranks_tensor.reshape(3,3,3).detach().numpy())*2

In [ ]:
threshold = np.array(action_count).mean() - 2*np.array(action_count).std()
action_CT = np.array(action_count).reshape(3,3,3)

In [ ]:
import plotly.graph_objs as go
import numpy as np

xs = np.arange(3)
ys = np.arange(3)
zs = np.arange(3)


uncertain_threshold = np.argwhere(action_CT <= threshold)
certain_threshold = np.argwhere(action_CT > threshold)

# Certain points
trace1 = go.Scatter3d(x=xs[certain_threshold[:, 0]],
                      y=ys[certain_threshold[:, 1]],
                      z=zs[certain_threshold[:, 2]],
                      mode='markers',
                      name='Certain Area',
                      marker=dict(
                          size=scale[certain_threshold[:, 0],
                                           certain_threshold[:, 1],
                                           certain_threshold[:, 2]],  # directly use rank_tensor as size
                          symbol='circle',
                          color=np.log(proxy_mean[certain_threshold[:, 0],
                                                            certain_threshold[:, 1],
                                                            certain_threshold[:, 2]]), # apply logarithm on color
                          colorbar=dict(thickness=10, ticklen=4, x=1),
                          colorscale='Viridis',   # choose a colorscale
                          opacity=0.8,
                          line=dict(color='Black', width=1)
                        ),
                      hovertemplate="Action 1: %{x}<br>Action 2: %{y}<br>Action 3: %{z}<br>LogSoftQ: %{marker.color}<extra></extra>",
                    )
# Uncertain points
trace2 = go.Scatter3d(x=xs[uncertain_threshold[:, 0]],
                      y=ys[uncertain_threshold[:, 1]],
                      z=zs[uncertain_threshold[:, 2]],
                      mode='markers',
                      name='Uncertain Area',
                      marker=dict(
                          size=10,
                          symbol='square',
                          color='rgb(255,0,0)',                # set color to red
                          opacity=0.8,
                          line=dict(color='Black', width=1)
                        ),
                      showlegend=True,
                      hovertemplate="Action 1: %{x}<br>Action 2: %{y}<br>Action 3: %{z}<br><extra></extra>",
                    )
layout = go.Layout(height=800, width=800, title='3D Heatmap',
                   scene=dict(
                              xaxis = dict(title='Action 1'),
                              yaxis = dict(title='Action 2'),
                              zaxis = dict(title='Action 3')
                             ),
                  )



fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()

In [ ]:
# distinctive, artist, research center

In [ ]:
create_prompt(1,0,0,get_prompt(3))

In [ ]:
prediction = pipe([create_prompt(1,1,1,get_prompt(random.randint(0, 19)))]).images
plt.imshow(prediction[0])
plt.axis('off')  
plt.show()